## 1. 다음영화 - 이미지 크롤링

- 2020년도 영화순위 이미지 가져오기
- res = requests.get(url)
- res.text : 문자열 형식으로 응답 content 추출
- res.content : 바이트열 형식을 응답 content 추출

In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://search.daum.net/search?w=tot&q=2020%EB%85%84%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84&DA=MOR&rtmaxcoll=MOR"
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

images = soup.find_all("img", class_="thumb_img")

for idx, image in enumerate(images):
    img_url = image["src"]
    #print(img_url)
    
    img_res = requests.get(img_url)
    img_res.raise_for_status()
    
    with open("data4/movie1/movie{}.jpg".format(idx+1),"wb") as f :
        f.write(img_res.content)
    



- 2020년 상위 5개 이미지만 가져오기

In [9]:
for idx, image in enumerate(images):
    img_url = image["src"]
    #print(img_url)
    
    img_res = requests.get(img_url)
    img_res.raise_for_status()
    
    with open("data4/movie2/movie{}.jpg".format(idx+1),"wb") as f :
        f.write(img_res.content)
    
    if idx>=4 : break

- 2015~2020년까지 상위 5개 영화 가지고 오기

In [18]:
for year in range(2015, 2021):
    url = "https://search.daum.net/search?w=tot&q={}년영화순위&DA=MOR&rtmaxcoll=MOR".format(year)
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    images = soup.find_all("img", class_="thumb_img")

    for idx, image in enumerate(images):
        img_url = image["src"]
    
        img_res = requests.get(img_url)
        img_res.raise_for_status()
    
        with open("data4/movie3/{} movie{}.jpg".format(year,idx+1),"wb") as f :
            f.write(img_res.content)
    
        if idx>=4 : break

## 2. 코스피 시가총액 순위

In [39]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
wb = Workbook()
ws = wb.active

title = "N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE	토론실".split('\t')
ws.append(title)


for page in range(1,5):
    res = requests.get(url+str(page))
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    data_rows = soup.find("table", attrs={"class":"type_2"}).find(("tbody")).find_all("tr")
    #print(len(data_rows))
    for row in data_rows:
        cols = row.find_all("td")
        
        if len(cols)<=1 : continue
        
        data = [col.get_text().strip() for col in cols]
        ws.append(data)

wb.save("data4/시가총액.xls")

## 3. 과제
### 3-1) 네이버에서 오늘 광주(경기도)의 날씨정보를 가져오기
[오늘의 날씨]   
맑음, 어제보다 4˚ 높아요   
현재 온도 :27℃ (최저 21˚/최고 27˚)   
강수확률 : 오전강수확률 30% / 오후강수확률 20%   
미세먼지 : 29㎍/㎥좋음   
초미세먼지 : 20㎍/㎥보통   
오존지수 : 0.064ppm보통   


In [30]:
import requests
from bs4 import BeautifulSoup

url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EC%98%A4%EB%8A%98%EB%82%A0%EC%94%A8"
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

# 오늘의 날씨
print("[오늘의 날씨]")
temp = soup.find("p",class_="cast_txt").get_text()
print(temp)

#현재온도
temp_now = soup.find("p",class_="info_temperature").get_text()
temp_n = temp_now[:2]+temp_now[-2:-3:-1]
min_max = soup.find("span",class_="merge").get_text()
print("현재온도 : {}(최저{}/최고{})".format(temp_n,min_max[1:4],min_max[5:8]))

#강수확률
morning = soup.find("span", class_="point_time morning").get_text().strip()
afternoon = soup.find("span", class_="point_time afternoon").get_text().strip()
print("강수확률 : 오전{} / 오후{}".format(morning, afternoon))

#미세먼지 
print('미세먼지 :',soup.find("dd", class_="lv1").get_text())

#초미세먼지 : 20㎍/㎥보통
print('초미세먼지 :',soup.find_all("dd", class_="lv1")[1].get_text())

#오존지수 : 0.064ppm보통
print('오존지수 :',soup.find_all("dd", class_="lv1")[2].get_text())

[오늘의 날씨]
구름많음, 어제보다 1˚ 높아요
현재온도 : 22℃(최저17˚/최고25˚)
강수확률 : 오전강수확률 30% / 오후강수확률 80%
미세먼지 : 25㎍/㎥좋음
초미세먼지 : 5㎍/㎥좋음
오존지수 : 0.02ppm좋음


In [55]:
# 교수님 답 _함수ver
import requests
from bs4 import BeautifulSoup

def scrape_weather():
    url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EC%98%A4%EB%8A%98%EB%82%A0%EC%94%A8"
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    
    print("[오늘의 날씨]")
    # 맑음, 어제보다 4˚ 높아요
    today = soup.find("p",class_="cast_txt").get_text()
    print('-',today)

    # 현재 온도 :27℃ (최저 21˚/최고 27˚)
    todaytemp = soup.find("p",class_="info_temperature").get_text().replace("도씨","")
    mintemp = soup.find("span",class_="min").get_text()
    maxtemp = soup.find("span",class_="max").get_text()
    print("- 현재 온도 : {} (최저 {}/최고 {})".format(todaytemp,mintemp,maxtemp))
    
    # 강수확률 : 오전강수확률 30% / 오후강수확률 20%
    morning_rain_rate = soup.find("span", class_="point_time morning").get_text().strip()
    afternoon_rain_rate = soup.find("span", class_="point_time afternoon").get_text().strip()
    print("- 강수확률 : 오전{} / 오후{}".format(morning_rain_rate, afternoon_rain_rate))
    
    #미세먼지 : 29㎍/㎥좋음, 초미세먼지 : 20㎍/㎥보통, 오존지수 : 0.064ppm보통
    dust = soup.find("dl", class_="indicator")
    print('-',dust.find_all("dt")[0].get_text(),":",dust.find_all("dd")[0].get_text())
    print('-',dust.find_all("dt")[1].get_text(),":",dust.find_all("dd")[1].get_text())
    print('-',dust.find_all("dt")[2].get_text(),":",dust.find_all("dd")[2].get_text())
    
scrape_()

[오늘의 날씨]
- 구름많음, 어제보다 0˚ 높아요
- 현재 온도 : 23℃  (최저 17˚/최고 25˚)
- 강수확률 : 오전강수확률 30% / 오후강수확률 80%
- 미세먼지 : 25㎍/㎥좋음
- 초미세먼지 : 5㎍/㎥좋음
- 오존지수 : 0.02ppm좋음


### 3-2) 네이버뉴스->it/과학->it일반 분야의 상위 3개 뉴스 크롤링 하기

[IT 뉴스]
1. 피 안 보고 혈당 측정
  (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=032&aid=0003095126 )
2. 갤폴드3 중고로 샀는데 “반납하세요” 문자…황당 사연, 무슨일?
  (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=016&aid=0001880910 )
3. 中, 미성년자 온라인게임 이용 주당 3시간으로 제한
  (링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=092&aid=0002232128 )

In [74]:
import requests
from bs4 import BeautifulSoup

url = "https://job.seoul.go.kr/www/job_offer_info/JobOfferInfo.do?method=selectJobOfferInfo"
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

soup.find_all('tbody')

#for b in body :
#    print(b.get_text())

#esthers = soup.find_all('table',border="1")

#for esther in esthers:
#    esther = esthers.td.get_text()
#    print(esther)


[]

In [29]:
url='https://www.jobkorea.co.kr/recruit/joblist?menucode=duty#anchorGICnt_1'
res = requests.get(url)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

soup.find('div',class_="tplList tplJobList")


<!--  EnableViewStateMac="false"  --><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>잡코리아 - JOBKOREA</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=no, minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
<style type="text/css">
		/* common */
		html, body, div, p, h1, img, ul, li, span, strong, a {margin:0; padding:0; line-height:0;font-family: 'Apple SD Gothic Neo', Arial, sans-serif; letter-spacing: -0.5px;}
		img {vertical-align:top; border:0;}
		ul, li {list-style:none;}
		a {text-decoration:none;}
		hr {display:none;}

		.skip {position:absolute; left:-9999px; top:-9999px; height:1px; width:1px;}

		/* user */
		.logoArea,.contents,.copy {width:100%; margin:0 auto;}
		.contents {width: 100%; max-width: 430px; padding: 20px 20px 0; box-sizing: border-box;}
		.logoArea {hei

In [31]:
url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=230"
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
res = requests.get(url, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")

print("[IT뉴스]")
print()
for i in range(3):
    headline = soup.find_all("dt", class_="photo")
    title = headline[i].img["alt"]
    link = headline[i].a["href"]
    print('{}. {}(링크 : {} )'.format(i+1,title,link))

[IT뉴스]

1. 1차접종자 3천만명 넘어…접종률 60% 임박(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=092&aid=0002232789 )
2. 올해 처음 메타버스로 열린 '구글 인디게임 페스티벌' Top3 선정(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=011&aid=0003958908 )
3. 한컴, 인도 기업과 손잡고 개발자 양성·수급 나선다(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=029&aid=0002695458 )


In [37]:
def create_soup(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"}
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    return soup

In [53]:
def print_news(idx, title, link) :
    print('{}.{}'.format(idx+1, title))
    print('(링크 : '+link+' )')

In [54]:
def scrape_it_news():
    print("[IT뉴스]")
    url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=230"
    soup = create_soup(url)
    
    news_list = soup.find("ul", class_="type06_headline").find_all("li",limit=3)
    for idx, news in enumerate(news_list) :
        a_idx = 0
        img = news.find("img")
        
        # img 태그가 있으면 1번째 a태그의 정보를 사용
        if img : 
            a_idx =1
        a_tag = news.find_all("a")[a_idx]
        title = a_tag.get_text().strip()
        link = a_tag["href"]
        print_news(idx, title, link)
scrape_it_news()

[IT뉴스]
1.국민지원금 사용처 '네이버·카카오 지도'에서 본다
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=008&aid=0004641159 )
2."네이버·카카오맵서 코로나 국민지원금 사용처 확인하세요"(종합)
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=001&aid=0012644259 )
3."추석 연휴에도 쏘카 타세요"···대여료 최대 78% 할인
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=011&aid=0003958938 )


In [56]:
scrape_weather()
scrape_it_news()

[오늘의 날씨]
- 구름많음, 어제보다 0˚ 높아요
- 현재 온도 : 23℃  (최저 17˚/최고 25˚)
- 강수확률 : 오전강수확률 30% / 오후강수확률 80%
- 미세먼지 : 25㎍/㎥좋음
- 초미세먼지 : 5㎍/㎥좋음
- 오존지수 : 0.02ppm좋음
[IT뉴스]
1.국민지원금 사용처 '네이버·카카오 지도'에서 본다
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=008&aid=0004641159 )
2."네이버·카카오맵서 코로나 국민지원금 사용처 확인하세요"(종합)
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=001&aid=0012644259 )
3."추석 연휴에도 쏘카 타세요"···대여료 최대 78% 할인
(링크 : https://news.naver.com/main/read.naver?mode=LS2D&mid=shm&sid1=105&sid2=230&oid=011&aid=0003958938 )


### 3-3) 만개의 레시피 사이트에서 명예의 전당 상위 5페이지 레시피 정보(제목, 링크) 가지고 오기   
[명예의 전당]   
닭봉간장조림~ 십년째 만들어 먹는 양념 공유해요! https://www.10000recipe.com/recipe/6940325   
백파더 백종원 에그치즈토스트 ~ 간단한데 맛은 최고! https://www.10000recipe.com/recipe/6939543   
..생략....

In [190]:
for i in range(5):
    if i==0 : url = "https://www.10000recipe.com/issue/view.html?cid=9999scrap&types="
    else : "https://www.10000recipe.com/issue/view.html?cid=9999scrap&types={}".format(i)

    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    thumbnails = soup.find_all("a", class_="thumbnail")
    for thumbnail in thumbnails :
        title = thumbnail.span.get_text()
        link = "https://www.10000recipe.com/"+thumbnail["href"]
        print(title, link)

닭봉간장조림~ 십년째 만들어 먹는 양념 공유해요! https://www.10000recipe.com//recipe/6940325
백파더 백종원 에그치즈토스트 ~ 간단한데 맛은 최고! https://www.10000recipe.com//recipe/6939543
♥[신혼밥상] 5분 반찬 상추겉절이 https://www.10000recipe.com//recipe/6929388
집밥으로 술안주로~ 닭갈비 레시피 끝판왕!! 파는 닭갈비 비법을 담았슴돠ㅋ~^^ https://www.10000recipe.com//recipe/6928209
즉석 목살 양념구이~ 달큰한 양념소스가 정말 맛있습니다. https://www.10000recipe.com//recipe/6925795
[아이반찬] 꼬들꼬들 식감이 좋은 맵지않은 오이무침 https://www.10000recipe.com//recipe/6923484
차돌박이 된장찌개 레시피ㅣ고깃집에서 먹던 바로 그 맛!ㅣ https://www.10000recipe.com//recipe/6922749
새우 부추전ㅣ집들이용으로도 손색 없는! https://www.10000recipe.com//recipe/6922700
치킨 스테이크 만들기ㅣ굽네가 부럽지 않은ㅣ https://www.10000recipe.com//recipe/6922190
간단한 술안주로 최고인 버터갈릭새우 만들기 https://www.10000recipe.com//recipe/6918977
맛있는 밑반찬 백종원 가지볶음 https://www.10000recipe.com//recipe/6917883
♥[신혼밥상] 10분 반찬! 메추리알조림 https://www.10000recipe.com//recipe/6916408
밥도둑 반찬 고추장 달걀조림 만들기!TV 알토란 레시피 https://www.10000recipe.com//recipe/6915088
감자어묵국 보들보들하게 즐기는 간단국요리 https://www.10000recipe.com//recipe/6914749
버섯으로